# Hydrogenic computations

In [1]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-7792779517658118884\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-9799258915683058178\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-7792779517658118884\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Perhaps, the simplest atomic computations can be made for hydrogenic ions. If we consider, for example, hydrogen-like argon $(Z=18)$, let us first determine and compare the energies of the $1s$ and $2p$ levels from the (non-relativistic) Schroedinger equation with those from Dirac's relativistic equation by

In [2]:
Z     = 18.0
e1s   = HydrogenicIon.energy(Shell("1s"), Z)           

  Energy for shell 1s is [in eV]: -4.40824454e+03


-162.0

In [3]:
e2p   = HydrogenicIon.energy(Shell("2p"), Z)
e2p_1 = HydrogenicIon.energy(Subshell("2p_1/2"), Z)   
e2p_3 = HydrogenicIon.energy(Subshell("2p_3/2"), Z)

  Energy for shell 2p is [in eV]: -1.10206113e+03
  Energy for subshell 2p_1/2 is [in eV]: -1.10805754e+03
  Energy for subshell 2p_3/2 is [in eV]: -1.10325210e+03


-40.54376720995463

While the (one-electron) energies are displayed by the function  `HydrogenicIon.energy()` in the default units (here eV), and as could be overwritten by the user, all **computations are internally performed and returned always in atomic units**, if not stated otherwise in the description of some particular function. This is easily seen from the output above and below for variables `e2p`, `e2p_1`, `e2p_3`: 

In [4]:
(e2p, e2p_1, e2p_3) 

(-40.5, -40.720363843040175, -40.54376720995463)

The units of energies, rates and several other physical properties can be however quite easily converted among each other, for instance, by:

In [5]:
e1s_eV = convertUnits("energy: from atomic to eV", e1s)
e1s_au = convertUnits("energy: from eV to atomic", e1s_eV)
(e1s, e1s_eV, e1s_au)

(-162.0, -4408.24453524, -162.0)

... and similarly also for other energy units as well as for other physical entities; cf. `? convertUnits()`.

From these energies, we can easily compute the fine-structure splitting of the $2p$ level into the $2p_{1/2}$ and $2p_{3/2}$ fine-structure levels for hydrogen-like argon from above:

In [6]:
e13 = e2p_1 - e2p_3
e13_eV = convertUnits("energy: from atomic to eV", e13)

-4.805439152723064


Apart from the (single-electron) energies, we can generate also the radial orbitals, i.e. the $P(r)$ in the non-relativistic theory or  $P(r)$ and $Q(r)$  in the relativistic theory. However, the general (and analytically well-known) solutions of Dirac's equation are presently not implemented in full detail. Instead, a good approximation is obtained by just applying the **kinetic-balance condition** to the non-relativistic $P^\mathrm{\,(nonrel)} (r)$  function and, then, by re-normalizing the large and small components ($P$ and $Q$) together. 
        
The non-relativistic radial orbitals $P(r)$ can be obtained either for a single $r$-value, for a list of $r$-values as well as for all $r$-values on a given radial grid. -- We here apply an exponential grid to make further use of these functions in computing various expectation values:


In [7]:
grid   = Radial.Grid("grid: exponential")
Pnr_1s = HydrogenicIon.radialOrbital(Shell("1s"), Z, grid)
Pnr_2p = HydrogenicIon.radialOrbital(Shell("2p"), Z, grid)

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines with break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


390-element Array{Float64,1}:
 0.0                   
 1.276274684307269e-13 
 3.291998155158454e-12 
 1.7395704550687125e-11
 4.9276751188999534e-11
 9.739045103145474e-11 
 1.494525618906226e-10 
 1.872027746796344e-10 
 2.115980901879572e-10 
 2.760306684336958e-10 
 3.9832478518661183e-10
 5.760410942349026e-10 
 7.864450578984476e-10 
 ⋮                     
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   
 0.0                   

Here, the exponential tails of the radial orbitals are simply set to zero if $|P| < 1.0e-15$ (and similarly for $|Q|$).

All these (radial) orbitals can be plotted also by the standard plot procedures; within JAC, however, special plotting functions are provided only for (relativistic) radial Orbital(s) which carry all their information by using the `struct Orbital`:

In [8]:
? Orbital

search: Orbital PhotoRecombination MultipolePolarizibility



`struct  Radial.Orbital`       ... defines a type for a single-electron radial orbital function with a large and small component, and which can refer to         either the standard or an explicitly given grid due to the logical flag useStandardGrid. Bound-state orbitals with energy < 0 are          distinguished from free-electron orbitals by the flag isBound.

```
+ subshell        ::Subshell          ... Relativistic subshell.
+ isBound         ::Bool              ... Logical flag to distinguish between bound (true) and free-electron orbitals (false).
+ useStandardGrid ::Bool              ... Logical flag for using the standard grid (true) or an explicitly given grid (false).
+ energy          ::Float64           ... Single-electron energies of bound orbitals are always negative.
+ P               ::Array{Float64,1}  ... Large and ..
+ Q               ::Array{Float64,1}  ... small component of the radial orbital.
+ Pprime          ::Array{Float64,1}  ... dP/dr.
+ Qprime          ::Array{Float64,1}  ... dQ/dr.
+ grid            ::Array{Float64,1}  ... explic. defined radial grid array for P, Q, if StandardGrid = false.
```

---

`Radial.Orbital(subshell::Subshell, energy::Float64)`       ... constructor for given subshell and energy, and where useStandardGrid is set to true; the grid must be defined          explicitly and neither the large and small components nor their derivatives are yet defined in this case.

---

`Radial.Orbital(label::String, energy::Float64)`       ... constructor for given string identifier and energy, and where useStandardGrid is set to true; the grid must be          defined explicitly and neither the large and small components nor their derivatives are yet defined in this case.



To display the non-relativistic  `Pnr_1s`$(r)$  orbitals, we need first internally to define a standard grid as well as such an (instance of the) Orbital by using its standard constructor, and by setting the small  $Q(r)$ component as well as $dP/dr$ and $dQ/dr$ simply to zero:

In [9]:
setDefaults("standard grid", grid)
Qnr_1s   = Pprime = Qprime = zeros( length(Pnr_1s) )
nrOrb_1s = Orbital( Subshell("1s_1/2"), true, true, -162.0, Pnr_1s, Qnr_1s, Pprime, Qprime, Radial.Grid())

Re-define the standard grid with 390 grid points.


Bound-state orbital 1s_1/2 with energy -162.0 a.u. is defined with 390 (grid) points on the standard grid: 
Large component P: [0.0, 3.25741e-6, 1.65436e-5, 3.80295e-5, 6.40058e-5, 8.9982e-5, 0.000111468, 0.000124753, 0.000132633, 0.000151486  …  0.000316219, 0.000342972, 0.000386236, 0.000438541, 0.000490846, 0.000534109, 0.00056086, 0.000576726, 0.000614688, 0.000676079]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Small component Q: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Pprime:            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Qprime:            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Defined

# **The following part of this nootebook is still under construction.**

We could also compute a relativistic $1s_{1/2}$ orbital on the given grid and plot the two $1s$-orbitals together; while the large components of these orbitals coincide of course, due to its special construction via the non-relativistic orbitals, the small compoments differ. Note that one need to use the standard REPL in order to display these functions in some separate window.

In [10]:
rOrb_1s = HydrogenicIon.radialOrbital(Subshell("1s_1/2"), Z, grid)
# JAC.plot("radial orbitals: large", Orbital[nrOrb_1s, rOrb_1s], grid; N = 230)
# JAC.plot("radial orbitals: small", Orbital[nrOrb_1s, rOrb_1s], grid; N = 230) 

  Energy for subshell 1s_1/2 is [in eV]: -4.42742470e+03
HydrogenicIon.radialOrbital():  for subshell 1s_1/2 : norm-before = 0.9999999999999974, norm-after = 1.0000000000000004


┌ Warning: radialOrbital():: Q[:] = zero everywhere; kinetic-balance not yet defined for Gauss-Legendre grids.
└ @ JAC.HydrogenicIon /home/fritzsch/fri/JAC.jl/src/module-HydrogenicIon.jl:102


Bound-state orbital 1s_1/2 with energy -162.70485811496013 a.u. is defined with 390 (grid) points on an explicitly-defined grid: 
Large component P: [0.0, 3.25741e-6, 1.65436e-5, 3.80295e-5, 6.40058e-5, 8.9982e-5, 0.000111468, 0.000124753, 0.000132633, 0.000151486  …  0.000316219, 0.000342972, 0.000386236, 0.000438541, 0.000490846, 0.000534109, 0.00056086, 0.000576726, 0.000614688, 0.000676079]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Small component Q: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Pprime:            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Qprime:            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  ...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

We can use these orbitals to quickly evaluate some overlap integrals and $\langle r^k \rangle$ expectation values; for this, we make use of the non-relativistic $2p$ radial orbital `Pnr_2p` from above an compute the normalization  and $\langle r \rangle$ expectation values:

In [11]:
N_1s = RadialIntegrals.overlap(Pnr_1s, Pnr_1s, grid)
N_2p = RadialIntegrals.overlap(Pnr_2p, Pnr_2p, grid)

ErrorException: stop a

For the sake of simplicity, all hydrogenic computations are performed for a point-like nucleus. For many-electron computations, in constrast, a more realistic nucleus can be taken into accout by selecting a proper nuclear model. See the tutorial: Define the nuclear model parameters   or   `? Nuclear.Model`  ... for further details.

In [12]:
 ? Nuclear.Model

`struct  Nuclear.Model`  ... defines a type for the nuclear model, i.e. for its form and parameters.

```
+ Z        ::Float64         ... nuclear charge
+ model    ::String          ... identifier of the nuclear model: {"Fermi", "Point", "Uniform"}
+ mass     ::Float64         ... atomic mass
+ radius   ::Float64         ... (root-mean square) radius of a uniform or Fermi-distributed nucleus
+ spinI    ::AngularJ64      ... nuclear spin I, must be >= 0
+ mu       ::Float64         ... magnetic dipole moment in Bohr magnetons
+ Q        ::Float64         ... electric qadrupole moment
```

---

`Nuclear.Model(Z::Real)`       ... to specify a Fermi-type nucleus with charge Z, and where the nuclear spin and nuclear moments are all set to zero.

---

`Nuclear.Model(Z::Real, model::String)`       ... to specify a nucleus with charge Z, model = {"Fermi", "point", "uniform"}, and where the nuclear spin and          nuclear moments are all set to zero.

---

`Nuclear.Model(gui::Guint; model::Nuclear.Model=Model(36.0))`       ... constructor to re-fine a nuclear model by a graphical user interface, and which may starts from          the parameters of a given model.
